This is a short demonstration of a Qwen finetuned model for (French) parlamentarian debates.

In [1]:
!uv pip install vllm

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
nvidia-cusolver-cu12 ------------------------------ 14.47 MiB/150.90 MiB
nvidia-cufft-cu12 ------------------------------ 14.61 MiB/190.95 MiB
nvidia-nccl-cu12 ------------------------------ 14.54 MiB/191.99 MiB
nvidia-cusparse-cu12 ------------------------------ 14.30 MiB/206.53 MiB
vllm       ------------------------------ 14.47 MiB/368.67 MiB
nvidia-cublas-cu12 ------------------------------ 14.56 MiB/374.93 MiB
nvidia-cudnn-cu12 ------------------------------ 14.58 MiB/544.54 MiB
⠹ Preparing packages... (39/57)
sympy      ------------------------------ 4.45 MiB/6.01 MiB
nvidia-nvjitlink-cu12 ------------------------------ 14.28 MiB/18.83 MiB
nvidia-cuda-nvrtc-cu12 ------------------------------ 14.39 MiB/22.55 MiB
llvmlite   ------------------------------ 14.70 MiB/40.40 MiB
nvidia-curand-cu12 ------------------------------ 14.74 MiB/53.67 MiB
ray        ------------------------------ 13.34 MiB/66.83 MiB


In [2]:
from vllm import LLM, SamplingParams

llm = LLM("Pclanglais/Qwen-Debate",
          max_model_len=16000)

INFO 08-05 07:59:03 [__init__.py:235] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

INFO 08-05 07:59:21 [config.py:1604] Using max model len 16000
INFO 08-05 07:59:22 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

WARNING 08-05 07:59:29 [__init__.py:2899] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized


In [3]:
sampling_params = SamplingParams(
    temperature=0.2,           # Truly greedy decoding (deterministic)
    max_tokens=4000,
    stop=["<|im_end|>"],  # Match your EOS token
    skip_special_tokens=False
)

prompts = ["""<|im_start|>user
French parliament session from 2025-03-11\n\nGeneral topic: Questions au gouvernement > Modèle social français\n\nSpecific topic:Questions au gouvernement > Modèle social français\n\nSpeech n°3656775 from Jérôme Guedj (Representative of the center-left (socialist)) :\n\nLe soutien à l’Ukraine et à une certaine idée de l’Europe, désormais autonome pour ce qui relève de sa sécurité, doit en effet être sans faille. Sans faille, mais pas sans raison. L’effort de défense ne peut servir de cheval de Troie à une fragilisation de notre modèle social. Au contraire : le maintien de notre modèle social est la condition d’un nécessaire patriotisme républicain. Ayons à l’esprit les mots du général de Gaulle en juin 1942…\n\nSpeech n°3656776 from Laurent Jacobelli (Representative of the far right) :\n\nVous n’allez pas vous y mettre !\n\nSpeech n°3656777 from Vincent Descœur (Representative of the republican right) :\n\nLe général fait désormais l’unanimité
<|im_end|>\n<|im_start|>assistant
"""]

outputs = llm.generate(prompts, sampling_params)
generated_texts = [output.outputs[0].text for output in outputs]

print(generated_texts[0])

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<proposal>
<proposal_name>French Social Model and National Defense</proposal_name>
<proposal_name_original>Modèle social français et défense nationale</proposal_name_original>

</proposal>

<speech identifier="3656775" name="Jérôme Guedj">
<stance>Assertive</stance>
<epistemic_frame>Historical reference ["les mots du général de Gaulle en juin 1942"]</epistemic_frame>
<epistemic_frame>Common sense ["le maintien de notre modèle social est la condition d’un nécessaire patriotisme républicain"]</epistemic_frame>
<emotion>Neutral</emotion>
<debate_adherence>Respectful</debate_adherence>
<audience>General public</audience>
<audience>Government</audience>
<figure_of_speech>Antithesis ["Sans faille, mais pas sans raison."]</figure_of_speech>
<figure_of_speech>Parallelism ["Sans faille, mais pas sans raison. L’effort de défense ne peut servir de cheval de Troie à une fragilisation de notre modèle social. Au contraire : le maintien de notre modèle social est la condition d’un nécessaire patrioti